In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from metal.mmtl.trainer import MultitaskTrainer
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.slicing.slice_model import SliceModel

In [3]:
SEED = 1

### Initialize normal payloads

In [4]:
task_kwargs = {
    "dl_kwargs": {"batch_size": 8},
    "freeze_bert":False,
    "bert_model": 'bert-base-cased',
    "max_len": 200,
    "attention": False
}
task_names = ["RTE"]

In [5]:
%%time

# Create tasks and payloads
tasks, payloads = create_glue_tasks_payloads(task_names, **task_kwargs)

Using random seed: 161224
Loading RTE Dataset



CPU times: user 13.5 s, sys: 1.1 s, total: 14.6 s
Wall time: 15.5 s


In [6]:
tasks, payloads

([ClassificationTask(name=RTE, loss_multiplier=1.00)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=train),
  Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=valid),
  Payload(RTE_test: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=test)])

### Initialize slice payloads

In [7]:
# Create tasks and payloads
task_kwargs.update({"slice_dict": {
    "RTE": ["dash_semicolon", "more_people", "BASE"]}
})
task_kwargs['attention'] = None

tasks_slice, payloads_slice = create_glue_tasks_payloads(
    task_names, **task_kwargs
)

Using random seed: 556582
Loading RTE Dataset



Added label_set with 2490/2490 labels for task RTE_slice:dash_semicolon:ind to payload RTE_train.
Added label_set with 1003/2490 labels for task RTE_slice:dash_semicolon:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:more_people:ind to payload RTE_train.
Added label_set with 64/2490 labels for task RTE_slice:more_people:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:BASE:ind to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:BASE:pred to payload RTE_train.
Added label_set with 277/277 labels for task RTE_slice:dash_semicolon:ind to payload RTE_valid.
Added label_set with 116/277 labels for task RTE_slice:dash_semicolon:pred to payload RTE_valid.
Added label_set with 277/277 labels for task RTE_slice:more_people:ind to payload RTE_valid.
Added label_set with 12/277 labels for task RTE_slice:more_people:pred to payload RTE_valid.
Added label_set with 277/277 labels for task RTE_slice:BA

In [8]:
tasks_slice, payloads_slice

([ClassificationTask(name=RTE, loss_multiplier=1.00),
  ClassificationTask(name=RTE_slice:dash_semicolon:ind, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:dash_semicolon:pred, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:more_people:ind, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:more_people:pred, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:BASE:ind, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:BASE:pred, loss_multiplier=0.17)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolon:ind': 'RTE_slice:dash_semicolon:ind', 'RTE_slice:dash_semicolon:pred': 'RTE_slice:dash_semicolon:pred', 'RTE_slice:more_people:ind': 'RTE_slice:more_people:ind', 'RTE_slice:more_people:pred': 'RTE_slice:more_people:pred', 'RTE_slice:BASE:ind': 'RTE_slice:BASE:ind', 'RTE_slice:BASE:pred': 'RTE_slice:BASE:pred'}], split=train),
  Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolo

In [9]:
payloads_slice[0].data_loader.dataset.labels['RTE_slice:BASE:pred'].view(1, -1)

tensor([[2, 1, 1,  ..., 2, 1, 2]])

In [10]:
payloads_slice[0].data_loader.dataset.labels['RTE_slice:BASE:ind'].view(1, -1)

tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.uint8)

In [11]:
payloads_slice[0].data_loader.dataset.labels['RTE_slice:dash_semicolon:pred'].view(1, -1)

tensor([[0, 0, 0,  ..., 0, 1, 0]])

In [12]:
payloads_slice[0].data_loader.dataset.labels['RTE_slice:dash_semicolon:ind'].view(1, -1)

tensor([[2, 2, 2,  ..., 2, 1, 2]], dtype=torch.uint8)

### Initialize and train baseline model 

In [13]:
model = MetalModel(tasks, seed=SEED, verbose=False)

In [14]:
%%time
trainer = MultitaskTrainer(seed=SEED)
trainer.train_model(
    model,
    payloads,
    checkpoint_metric="RTE/RTE_valid/RTE_gold/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=5e-5,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=30,
    progress_bar=True,
    checkpoint_best=True,
    checkpoint_cleanup=False,
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 312 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/vschen/metal-mmtl/logs/2019_04_23/00_47_39/config.json


[0.10 epo]: RTE:[RTE_train/RTE_gold/loss=6.90e-01, RTE_valid/RTE_gold/accuracy=5.45e-01] model:[train/all/loss=6.90e-01, train/all/lr=5.00e-05]
Saving model at iteration 0.10 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.545
[0.21 epo]: RTE:[RTE_train/RTE_gold/loss=7.19e-01, RTE_valid/RTE_gold/accuracy=4.77e-01] model:[train/all/loss=7.19e-01, train/all/lr=5.00e-05]
[0.31 epo]: RTE:[RTE_train/RTE_gold/loss=6.91e-01, RTE_valid/RTE_gold/accuracy=5.34e-01] model:[train/all/loss=6.91e-01, train/all/lr=5.00e-05]
[0.41 epo]: RTE:[RTE_train/RTE_gold/loss=7.20e-01, RTE_valid/RTE_gold/accuracy=5.49e-01] model:[train/all/loss=7.20e-01, train/all/lr=5.00e-05]
Saving model at iteration 0.41 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.549
[0.51 epo]: RTE:[RTE_train/RTE_gold/loss=6.71e-01, RTE_valid/RTE_gold/accuracy=5.99e-01] model:[train/all/loss=6.71e-01, train/all/lr=5.00e-05]
Saving model at iteration 0.51 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.599
[0.62 e

[1.03 epo]: RTE:[RTE_train/RTE_gold/loss=6.25e-01, RTE_valid/RTE_gold/accuracy=6.32e-01] model:[train/all/loss=6.25e-01, train/all/lr=5.00e-05]
[1.13 epo]: RTE:[RTE_train/RTE_gold/loss=5.52e-01, RTE_valid/RTE_gold/accuracy=6.53e-01] model:[train/all/loss=5.52e-01, train/all/lr=5.00e-05]
Saving model at iteration 1.13 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.653
[1.23 epo]: RTE:[RTE_train/RTE_gold/loss=5.85e-01, RTE_valid/RTE_gold/accuracy=6.46e-01] model:[train/all/loss=5.85e-01, train/all/lr=5.00e-05]
[1.33 epo]: RTE:[RTE_train/RTE_gold/loss=5.68e-01, RTE_valid/RTE_gold/accuracy=6.21e-01] model:[train/all/loss=5.68e-01, train/all/lr=5.00e-05]
[1.44 epo]: RTE:[RTE_train/RTE_gold/loss=5.92e-01, RTE_valid/RTE_gold/accuracy=6.61e-01] model:[train/all/loss=5.92e-01, train/all/lr=5.00e-05]
Saving model at iteration 1.44 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.661
[1.54 epo]: RTE:[RTE_train/RTE_gold/loss=5.81e-01, RTE_valid/RTE_gold/accuracy=6.35e-01] model:[tr

[2.05 epo]: RTE:[RTE_train/RTE_gold/loss=5.37e-01, RTE_valid/RTE_gold/accuracy=6.61e-01] model:[train/all/loss=5.37e-01, train/all/lr=5.00e-05]
[2.15 epo]: RTE:[RTE_train/RTE_gold/loss=3.62e-01, RTE_valid/RTE_gold/accuracy=6.90e-01] model:[train/all/loss=3.62e-01, train/all/lr=5.00e-05]
[2.26 epo]: RTE:[RTE_train/RTE_gold/loss=3.26e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=3.26e-01, train/all/lr=5.00e-05]
[2.36 epo]: RTE:[RTE_train/RTE_gold/loss=4.77e-01, RTE_valid/RTE_gold/accuracy=6.68e-01] model:[train/all/loss=4.77e-01, train/all/lr=5.00e-05]
[2.46 epo]: RTE:[RTE_train/RTE_gold/loss=4.62e-01, RTE_valid/RTE_gold/accuracy=6.64e-01] model:[train/all/loss=4.62e-01, train/all/lr=5.00e-05]
[2.56 epo]: RTE:[RTE_train/RTE_gold/loss=4.29e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=4.29e-01, train/all/lr=5.00e-05]
[2.67 epo]: RTE:[RTE_train/RTE_gold/loss=4.83e-01, RTE_valid/RTE_gold/accuracy=6.86e-01] model:[train/all/loss=4.83e-01, train/all/lr=5.

[3.08 epo]: RTE:[RTE_train/RTE_gold/loss=2.75e-01, RTE_valid/RTE_gold/accuracy=6.86e-01] model:[train/all/loss=2.75e-01, train/all/lr=5.00e-05]
[3.18 epo]: RTE:[RTE_train/RTE_gold/loss=2.67e-01, RTE_valid/RTE_gold/accuracy=6.46e-01] model:[train/all/loss=2.67e-01, train/all/lr=5.00e-05]
[3.28 epo]: RTE:[RTE_train/RTE_gold/loss=4.75e-01, RTE_valid/RTE_gold/accuracy=7.15e-01] model:[train/all/loss=4.75e-01, train/all/lr=5.00e-05]
Saving model at iteration 3.28 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.715
[3.38 epo]: RTE:[RTE_train/RTE_gold/loss=2.63e-01, RTE_valid/RTE_gold/accuracy=6.97e-01] model:[train/all/loss=2.63e-01, train/all/lr=5.00e-05]
[3.49 epo]: RTE:[RTE_train/RTE_gold/loss=2.71e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=2.71e-01, train/all/lr=5.00e-05]
[3.59 epo]: RTE:[RTE_train/RTE_gold/loss=4.10e-01, RTE_valid/RTE_gold/accuracy=6.61e-01] model:[train/all/loss=4.10e-01, train/all/lr=5.00e-05]
[3.69 epo]: RTE:[RTE_train/RTE_gold/loss=3.61

...

[25.03 epo]: RTE:[RTE_train/RTE_gold/loss=4.62e-01, RTE_valid/RTE_gold/accuracy=5.74e-01] model:[train/all/loss=4.62e-01, train/all/lr=5.00e-05]
[25.13 epo]: RTE:[RTE_train/RTE_gold/loss=4.11e-01, RTE_valid/RTE_gold/accuracy=5.81e-01] model:[train/all/loss=4.11e-01, train/all/lr=5.00e-05]
[25.23 epo]: RTE:[RTE_train/RTE_gold/loss=4.02e-01, RTE_valid/RTE_gold/accuracy=5.63e-01] model:[train/all/loss=4.02e-01, train/all/lr=5.00e-05]
[25.33 epo]: RTE:[RTE_train/RTE_gold/loss=5.06e-01, RTE_valid/RTE_gold/accuracy=5.92e-01] model:[train/all/loss=5.06e-01, train/all/lr=5.00e-05]
[25.44 epo]: RTE:[RTE_train/RTE_gold/loss=4.69e-01, RTE_valid/RTE_gold/accuracy=5.60e-01] model:[train/all/loss=4.69e-01, train/all/lr=5.00e-05]
[25.54 epo]: RTE:[RTE_train/RTE_gold/loss=4.10e-01, RTE_valid/RTE_gold/accuracy=6.10e-01] model:[train/all/loss=4.10e-01, train/all/lr=5.00e-05]
[25.64 epo]: RTE:[RTE_train/RTE_gold/loss=4.74e-01, RTE_valid/RTE_gold/accuracy=5.56e-01] model:[train/all/loss=4.74e-01, train/al

[26.05 epo]: RTE:[RTE_train/RTE_gold/loss=3.40e-01, RTE_valid/RTE_gold/accuracy=5.16e-01] model:[train/all/loss=3.40e-01, train/all/lr=5.00e-05]
[26.15 epo]: RTE:[RTE_train/RTE_gold/loss=4.87e-01, RTE_valid/RTE_gold/accuracy=5.63e-01] model:[train/all/loss=4.87e-01, train/all/lr=5.00e-05]
[26.26 epo]: RTE:[RTE_train/RTE_gold/loss=4.18e-01, RTE_valid/RTE_gold/accuracy=5.52e-01] model:[train/all/loss=4.18e-01, train/all/lr=5.00e-05]
[26.36 epo]: RTE:[RTE_train/RTE_gold/loss=4.44e-01, RTE_valid/RTE_gold/accuracy=5.63e-01] model:[train/all/loss=4.44e-01, train/all/lr=5.00e-05]
[26.46 epo]: RTE:[RTE_train/RTE_gold/loss=4.17e-01, RTE_valid/RTE_gold/accuracy=5.81e-01] model:[train/all/loss=4.17e-01, train/all/lr=5.00e-05]
[26.56 epo]: RTE:[RTE_train/RTE_gold/loss=5.04e-01, RTE_valid/RTE_gold/accuracy=5.23e-01] model:[train/all/loss=5.04e-01, train/all/lr=5.00e-05]
[26.67 epo]: RTE:[RTE_train/RTE_gold/loss=4.31e-01, RTE_valid/RTE_gold/accuracy=5.63e-01] model:[train/all/loss=4.31e-01, train/al

[27.08 epo]: RTE:[RTE_train/RTE_gold/loss=4.82e-01, RTE_valid/RTE_gold/accuracy=5.70e-01] model:[train/all/loss=4.82e-01, train/all/lr=5.00e-05]
[27.18 epo]: RTE:[RTE_train/RTE_gold/loss=4.55e-01, RTE_valid/RTE_gold/accuracy=5.45e-01] model:[train/all/loss=4.55e-01, train/all/lr=5.00e-05]
[27.28 epo]: RTE:[RTE_train/RTE_gold/loss=5.24e-01, RTE_valid/RTE_gold/accuracy=5.45e-01] model:[train/all/loss=5.24e-01, train/all/lr=5.00e-05]
[27.38 epo]: RTE:[RTE_train/RTE_gold/loss=4.75e-01, RTE_valid/RTE_gold/accuracy=5.52e-01] model:[train/all/loss=4.75e-01, train/all/lr=5.00e-05]
[27.49 epo]: RTE:[RTE_train/RTE_gold/loss=4.02e-01, RTE_valid/RTE_gold/accuracy=5.60e-01] model:[train/all/loss=4.02e-01, train/all/lr=5.00e-05]
[27.59 epo]: RTE:[RTE_train/RTE_gold/loss=4.22e-01, RTE_valid/RTE_gold/accuracy=5.67e-01] model:[train/all/loss=4.22e-01, train/all/lr=5.00e-05]
[27.69 epo]: RTE:[RTE_train/RTE_gold/loss=4.24e-01, RTE_valid/RTE_gold/accuracy=5.70e-01] model:[train/all/loss=4.24e-01, train/al

[28.10 epo]: RTE:[RTE_train/RTE_gold/loss=4.07e-01, RTE_valid/RTE_gold/accuracy=5.45e-01] model:[train/all/loss=4.07e-01, train/all/lr=5.00e-05]
[28.21 epo]: RTE:[RTE_train/RTE_gold/loss=4.33e-01, RTE_valid/RTE_gold/accuracy=5.74e-01] model:[train/all/loss=4.33e-01, train/all/lr=5.00e-05]
[28.31 epo]: RTE:[RTE_train/RTE_gold/loss=4.10e-01, RTE_valid/RTE_gold/accuracy=5.13e-01] model:[train/all/loss=4.10e-01, train/all/lr=5.00e-05]
[28.41 epo]: RTE:[RTE_train/RTE_gold/loss=4.30e-01, RTE_valid/RTE_gold/accuracy=5.20e-01] model:[train/all/loss=4.30e-01, train/all/lr=5.00e-05]
[28.51 epo]: RTE:[RTE_train/RTE_gold/loss=4.95e-01, RTE_valid/RTE_gold/accuracy=5.38e-01] model:[train/all/loss=4.95e-01, train/all/lr=5.00e-05]
[28.62 epo]: RTE:[RTE_train/RTE_gold/loss=4.86e-01, RTE_valid/RTE_gold/accuracy=5.67e-01] model:[train/all/loss=4.86e-01, train/all/lr=5.00e-05]
[28.72 epo]: RTE:[RTE_train/RTE_gold/loss=4.44e-01, RTE_valid/RTE_gold/accuracy=5.56e-01] model:[train/all/loss=4.44e-01, train/al

[29.03 epo]: RTE:[RTE_train/RTE_gold/loss=4.97e-01, RTE_valid/RTE_gold/accuracy=5.31e-01] model:[train/all/loss=4.97e-01, train/all/lr=5.00e-05]
[29.13 epo]: RTE:[RTE_train/RTE_gold/loss=5.02e-01, RTE_valid/RTE_gold/accuracy=5.56e-01] model:[train/all/loss=5.02e-01, train/all/lr=5.00e-05]
[29.23 epo]: RTE:[RTE_train/RTE_gold/loss=4.73e-01, RTE_valid/RTE_gold/accuracy=5.60e-01] model:[train/all/loss=4.73e-01, train/all/lr=5.00e-05]
[29.33 epo]: RTE:[RTE_train/RTE_gold/loss=3.80e-01, RTE_valid/RTE_gold/accuracy=5.38e-01] model:[train/all/loss=3.80e-01, train/all/lr=5.00e-05]
[29.44 epo]: RTE:[RTE_train/RTE_gold/loss=4.14e-01, RTE_valid/RTE_gold/accuracy=5.27e-01] model:[train/all/loss=4.14e-01, train/all/lr=5.00e-05]
[29.54 epo]: RTE:[RTE_train/RTE_gold/loss=4.40e-01, RTE_valid/RTE_gold/accuracy=5.63e-01] model:[train/all/loss=4.40e-01, train/all/lr=5.00e-05]
[29.64 epo]: RTE:[RTE_train/RTE_gold/loss=4.67e-01, RTE_valid/RTE_gold/accuracy=5.45e-01] model:[train/all/loss=4.67e-01, train/al

### Evaluate baseline slices

In [15]:
import copy
eval_payload = copy.deepcopy(payloads_slice[1])

# NOTE: we need to retarget slices to the original RTE head
for label_name in ['RTE_slice:dash_semicolon', 'RTE_slice:more_people', 'RTE_slice:BASE']:
    for slice_head_type in ['ind', 'pred']:
        eval_payload.retarget_labelset(f'{label_name}:{slice_head_type}', 'RTE')

labelset 'RTE_slice:dash_semicolon:ind' -> task 'RTE' (originally, RTE_slice:dash_semicolon:ind).
labelset 'RTE_slice:dash_semicolon:pred' -> task 'RTE' (originally, RTE_slice:dash_semicolon:pred).
labelset 'RTE_slice:more_people:ind' -> task 'RTE' (originally, RTE_slice:more_people:ind).
labelset 'RTE_slice:more_people:pred' -> task 'RTE' (originally, RTE_slice:more_people:pred).
labelset 'RTE_slice:BASE:ind' -> task 'RTE' (originally, RTE_slice:BASE:ind).
labelset 'RTE_slice:BASE:pred' -> task 'RTE' (originally, RTE_slice:BASE:pred).


In [16]:
eval_payload

Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolon:ind': 'RTE', 'RTE_slice:dash_semicolon:pred': 'RTE', 'RTE_slice:more_people:ind': 'RTE', 'RTE_slice:more_people:pred': 'RTE', 'RTE_slice:BASE:ind': 'RTE', 'RTE_slice:BASE:pred': 'RTE'}], split=valid)

In [17]:
model.score(eval_payload)

{'RTE/RTE_valid/RTE_gold/accuracy': 0.7148014440433214,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 0.48375451263537905,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.6206896551724138,
 'RTE/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.4187725631768953,
 'RTE/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.6666666666666666,
 'RTE/RTE_valid/RTE_slice:BASE:ind/accuracy': 0.5740072202166066,
 'RTE/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.7148014440433214}

### Initialize and train slice model

In [18]:
from metal.mmtl.slicing.tasks import convert_to_slicing_tasks
slicing_tasks = convert_to_slicing_tasks(tasks_slice)
slicing_tasks

Modifying RTE out_features from 2 -> 1
Modifying RTE_slice:dash_semicolon:ind out_features from 2 -> 1
Modifying RTE_slice:dash_semicolon:pred out_features from 2 -> 1
Modifying RTE_slice:more_people:ind out_features from 2 -> 1
Modifying RTE_slice:more_people:pred out_features from 2 -> 1
Modifying RTE_slice:BASE:ind out_features from 2 -> 1
Modifying RTE_slice:BASE:pred out_features from 2 -> 1


[BinaryClassificationTask(name=RTE, loss_multiplier=1.00, slice_head_type=None),
 BinaryClassificationTask(name=RTE_slice:dash_semicolon:ind, loss_multiplier=0.17, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:dash_semicolon:pred, loss_multiplier=0.17, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:more_people:ind, loss_multiplier=0.17, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:more_people:pred, loss_multiplier=0.17, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:BASE:ind, loss_multiplier=0.17, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:BASE:pred, loss_multiplier=0.17, slice_head_type=pred)]

In [19]:
slice_model = SliceModel(slicing_tasks, seed=SEED, verbose=False)

In [20]:
%%time
trainer = MultitaskTrainer(seed=SEED)
trainer.train_model(
    slice_model,
    payloads_slice,
    task_metrics=[
        "RTE/RTE_train/RTE_gold/loss", 
        "RTE/RTE_train/RTE_slice:BASE:ind/loss",
        "RTE/RTE_train/RTE_slice:dash_semicolon:ind/loss", 
        "RTE/RTE_train/RTE_slice:more_people:ind/loss",
        "RTE/RTE_train/RTE_slice:BASE:pred/loss",
        "RTE/RTE_train/RTE_slice:dash_semicolon:pred/loss", 
        "RTE/RTE_train/RTE_slice:more_people:pred/loss",
        
        "RTE/RTE_valid/RTE_gold/accuracy",
        "RTE/RTE_valid/RTE_slice:BASE:ind/accuracy", 
        "RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy", 
        "RTE/RTE_valid/RTE_slice:more_people:ind/accuracy",
        "RTE/RTE_valid/RTE_slice:BASE:pred/accuracy", 
        "RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy", 
        "RTE/RTE_valid/RTE_slice:more_people:pred/accuracy",
    ],
    checkpoint_metric="RTE/RTE_valid/RTE_gold/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=5e-5,
#     lr=1e-5,
#     l2=1e-3,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=30,
    progress_bar=True,
    checkpoint_best=True,
    checkpoint_cleanup=False
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 312 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/vschen/metal-mmtl/logs/2019_04_23/01_30_48/config.json


[0.10 epo]: RTE:[RTE_train/RTE_gold/loss=6.94e-01, RTE_valid/RTE_gold/accuracy=4.73e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.13e-01, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=5.74e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.17e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=6.12e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=8.44e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.31e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=6.67e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=1.09e-01, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=1.23e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=5.34e-01] model:[train/all/loss=2.16e-01, train/all/lr=5.00e-05]
Saving model at iteration 0.10 with best (max) score RTE/RTE_valid/RTE_go

[1.03 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.73e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=8.61e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=8.84e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.11e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.78e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=2.17e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=4.40e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=1.10e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=6.46e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.22e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] model:[train/all/loss=1.77e-01, train/all/lr=5.00e-05]
[1.13 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/acc

[1.95 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.73e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.40e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.82e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.09e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.52e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=2.12e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=3.99e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=1.09e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=5.81e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.07e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] model:[train/all/loss=1.64e-01, train/all/lr=5.00e-05]



[2.05 epo]: RTE:[RTE_train/RTE_gold/loss=6.92e-01, RTE_valid/RTE_gold/accuracy=4.73e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.45e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.82e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.04e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.86e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=1.55e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=3.33e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=1.14e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=6.46e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.02e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] model:[train/all/loss=1.64e-01, train/all/lr=5.00e-05]
[2.15 epo]: RTE:[RTE_train/RTE_gold/loss=6.91e-01, RTE_valid/RTE_gold/acc

...

[27.08 epo]: RTE:[RTE_train/RTE_gold/loss=4.18e-01, RTE_valid/RTE_gold/accuracy=5.96e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.03e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.93e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=3.24e-02, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.43e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=1.11e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.44e-02, RTE_valid/RTE_slice:more_people:pred/accuracy=2.50e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=1.05e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=5.75e-02, RTE_valid/RTE_slice:BASE:pred/accuracy=5.96e-01] model:[train/all/loss=9.50e-02, train/all/lr=5.00e-05]
[27.18 epo]: RTE:[RTE_train/RTE_gold/loss=4.37e-01, RTE_valid/RTE_gold/a

[28.0 epo]: RTE:[RTE_train/RTE_gold/loss=3.88e-01, RTE_valid/RTE_gold/accuracy=6.32e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.34e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.96e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=4.60e-02, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=6.21e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=1.83e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=9.90e-04, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=4.68e-02, RTE_valid/RTE_slice:BASE:pred/accuracy=6.32e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.27e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=5.00e-01] model:[train/all/loss=8.99e-02, train/all/lr=5.00e-05]



[28.10 epo]: RTE:[RTE_train/RTE_gold/loss=4.15e-01, RTE_valid/RTE_gold/accuracy=6.06e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.99e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.96e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=4.58e-02, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=6.12e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=1.82e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=1.07e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=6.08e-02, RTE_valid/RTE_slice:BASE:pred/accuracy=6.06e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=3.78e-03, RTE_valid/RTE_slice:more_people:pred/accuracy=2.50e-01] model:[train/all/loss=9.82e-02, train/all/lr=5.00e-05]
[28.21 epo]: RTE:[RTE_train/RTE_gold/loss=4.52e-01, RTE_valid/RTE_gold/a

[29.03 epo]: RTE:[RTE_train/RTE_gold/loss=3.93e-01, RTE_valid/RTE_gold/accuracy=5.96e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.66e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.96e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=5.70e-02, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.60e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=2.63e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=1.08e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=5.02e-02, RTE_valid/RTE_slice:BASE:pred/accuracy=5.99e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=7.22e-02, RTE_valid/RTE_slice:more_people:pred/accuracy=2.50e-01] model:[train/all/loss=9.44e-02, train/all/lr=5.00e-05]
[29.13 epo]: RTE:[RTE_train/RTE_gold/loss=3.86e-01, RTE_valid/RTE_gold/a

[29.95 epo]: RTE:[RTE_train/RTE_gold/loss=4.41e-01, RTE_valid/RTE_gold/accuracy=5.88e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.58e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.96e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=5.33e-02, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.60e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=2.02e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=8.58e-04, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=7.52e-02, RTE_valid/RTE_slice:BASE:pred/accuracy=5.88e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.79e-02, RTE_valid/RTE_slice:more_people:pred/accuracy=3.33e-01] model:[train/all/loss=1.05e-01, train/all/lr=5.00e-05]

[30.00 epo]: RTE:[RTE_train/RTE_gold/loss=5.07e-01, RTE_valid/RTE_gold/

#### Did we improve?

In [21]:
%%time
slice_model.score(payloads_slice[1])

CPU times: user 1.54 s, sys: 284 ms, total: 1.82 s
Wall time: 1.77 s


{'RTE/RTE_valid/RTE_gold/accuracy': 0.6678700361010831,
 'RTE_slice:dash_semicolon:ind/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 1.0,
 'RTE_slice:dash_semicolon:pred/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.6120689655172413,
 'RTE_slice:more_people:ind/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.9566787003610109,
 'RTE_slice:more_people:pred/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.75,
 'RTE_slice:BASE:ind/RTE_valid/RTE_slice:BASE:ind/accuracy': 1.0,
 'RTE_slice:BASE:pred/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.6462093862815884}

In [22]:
slice_model.score(eval_payload)

{'RTE/RTE_valid/RTE_gold/accuracy': 0.6678700361010831,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 0.4368231046931408,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.6120689655172413,
 'RTE/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.37906137184115524,
 'RTE/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.6666666666666666,
 'RTE/RTE_valid/RTE_slice:BASE:ind/accuracy': 0.5992779783393501,
 'RTE/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.6678700361010831}